In [1]:
from benzinga import financial_data
from benzinga import news_data

import pandas as pd
import numpy as np
import json
import inspect
import datetime

In [2]:
api_key = "318da1f2bee64e3caf1c22dfb78f70d1"
news = news_data.News(api_key)

In [3]:
# Optional:
# pagesize (int) - default is 15
# page (int) - default is 0
# display_output (str) - select from (full, abstract, headline)
# base_date (str) - "YYYY-MM-DD" The date to query for calendar data. Shorthand for date_from and date_to if they are the same. Defaults for latest.
# date_from (str) - "YYYY-MM-DD"
# date_to (str) - "YYYY-MM-DD"
# last_id (str) - The last ID to start paging from and sorted by and sorted by the last updated date.
# updated_since (str) - he last updated unix timestamp (UTC) to pull and sort by.
# publish_since (str) - The last publish unix timestamp (UTC) to pull and sort by.
# company_tickers (str)
# channel (str) - multiple channels separated by comma.

In [4]:
# def benzinga_call(ticker, fromdate, todate):
#     stories = news.news(display_output = 'full', company_tickers = ticker, pagesize = 100, date_from = fromdate, date_to = todate)
#     df = pd.DataFrame(stories)
#     df['created'] = pd.to_datetime(df['created']).dt.date
#     fromdate = (df.iloc[-1,2] - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    
#     one_month_be4_todate = (datetime.datetime.strptime(todate, '%Y-%m-%d') - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
#     while fromdate < one_month_be4_todate:
#         stories = news.news(display_output = 'full', company_tickers = ticker, pagesize = 100, date_from = fromdate, date_to = todate)
#         stories_df = pd.DataFrame(stories)
#         stories_df['created'] = pd.to_datetime(stories_df['created']).dt.date
#         df = pd.concat([df,stories_df]).drop_duplicates(subset=['id']).reset_index(drop=True)
#         fromdate = (df.iloc[-1,2] - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
#     return df

In [5]:
def benzinga_call(ticker, fromdate, todate):
    stories = news.news(display_output='full', company_tickers=ticker, pagesize=100, date_from=fromdate, date_to=todate)
    df = pd.DataFrame(stories)
    df['created'] = pd.to_datetime(df['created']).dt.date
    fromdate = (df.iloc[-1, 2] - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

    one_month_be4_todate = (datetime.datetime.strptime(todate, '%Y-%m-%d') - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
    last_request_fromdate = None
    while fromdate < one_month_be4_todate:
        if last_request_fromdate is not None and fromdate <= last_request_fromdate:
            fromdate = (datetime.datetime.strptime(last_request_fromdate, '%Y-%m-%d') + datetime.timedelta(days=15)).strftime('%Y-%m-%d')
            continue

        stories = news.news(display_output='full', company_tickers=ticker, pagesize=100, date_from=fromdate, date_to=todate)
        stories_df = pd.DataFrame(stories)
        stories_df['created'] = pd.to_datetime(stories_df['created']).dt.date
        df = pd.concat([df, stories_df]).drop_duplicates(subset=['id']).reset_index(drop=True)
        last_request_fromdate = fromdate
        fromdate = (df.iloc[-1, 2] - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    return df

In [6]:
ticker = 'AAPL'
fromdate = "2010-01-01"
todate = "2023-03-26"
df = benzinga_call(ticker, fromdate, todate)

2023-03-27 03:47:09 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb78f70d1&pageSize=100&displayOutput=full&dateFrom=2010-01-01&dateTo=2023-03-26&tickers=AAPL
2023-03-27 03:47:09 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb78f70d1&pageSize=100&displayOutput=full&dateFrom=2010-02-02&dateTo=2023-03-26&tickers=AAPL
2023-03-27 03:47:10 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb78f70d1&pageSize=100&displayOutput=full&dateFrom=2010-04-05&dateTo=2023-03-26&tickers=AAPL
2023-03-27 03:47:10 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb78f70d1&pageSize=100&displayOutput=full&dateFrom=2010-05-12&dateTo=2023-03-26&tickers=AAPL
2023-03-27 03:47:10 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb7

In [7]:
df.shape

(13274, 12)

In [9]:
df.to_csv('AAPL_benzinga_2010_to_2023_3_26.csv', index=False)